In [5]:
!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [6]:
import os
print(os.name)

posix


In [7]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import twitter_samples 

nltk.download('movie_reviews')
nltk.download('twitter_samples')

import pandas as pd
import numpy as np

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
stop_words = stopwords.words('english')
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/shijia_huang/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [25]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

In [14]:
url="https://www.dropbox.com/s/wt3dyh2thhkhmxg/qualtric_sentiment.csv?dl=1"
df = pd.read_csv(url)
df.head(9)

,review_id,sent_idx,sent_text,sentiment_human,length
0,2151,0,"I love the MDX, It blows away the Lexus for th...",1,93
1,2151,1,but doesn't have any of the features that are ...,0,69
2,2151,2,The Nav system & the entertainment system are ...,1,87
3,2151,3,It's the best luxury vehicle for the money.,1,44
4,2162,0,i love my mdx,1,13
5,2980,0,"After a year and two road trips, here are my g...",1,65
6,2980,1,"Pros:Great mileage (28 mpg on the road), comfo...",1,140
7,2980,2,"Cons:Interior finish uninspiring, fit and fini...",0,195
8,2980,3,We've had the car in for two fit and finish re...,0,52


In [20]:
# df.loc[df['length']>df['length'].mean(), 'is_short']=0
 

test_text_pos = []
test_text_neg = []
for index in range(int(len(df.index))):
    if df.at[index,'sentiment_human'] == 1:
        test_text_pos.append(df.at[index,'sent_text'])
    if df.at[index,'sentiment_human'] == 0:
        test_text_neg.append(df.at[index,'sent_text'])


print(len(test_text_pos))
print(len(test_text_neg))

if(test_text_neg[2] != df.at[8,'sent_text']):
    print("something went wrong")
else:
    print("we are cool")



2206
783
we are cool


In [15]:
def read_from_folder(path):
    file_names = []
    text_list = []
    for filename in os.listdir(path):
        if filename.endswith(".txt"): 
            file_names.append(path + filename)
            #print(filename)  
    for file in file_names:
        f = open(file,"r+")
        content = f.read()
        text_list.append(content)
        f.close()
    return text_list, file_names

In [16]:
train_text_neg, train_file_neg = read_from_folder("/Users/shijia_huang/Desktop/The Thing/aclImdb/train/neg/")

In [17]:
train_text_pos, train_file_pos = read_from_folder("/Users/shijia_huang/Desktop/The Thing/aclImdb/train/pos/")

In [122]:
# train_text_unsup, train_file_unsup = read_from_folder("/Users/shijia_huang/Desktop/The Thing/aclImdb/train/unsup/")

In [21]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [22]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [23]:
def get_tweets_for_model(cleaned_token_list):
    for tweet_tokens in cleaned_token_list:
        yield dict([token, True] for token in tweet_tokens)

In [26]:
positive_train_tweets = train_text_pos
negative_train_tweets = train_text_neg
#unsup_train_tweets = train_text_unsup 

positive_train_tweet_tokens = [word_tokenize(i) for i in positive_train_tweets]
negative_train_tweet_tokens = [word_tokenize(i) for i in negative_train_tweets]
#unsup_train_tweet_tokens = [word_tokenize(i) for i in unsup_train_tweets]

positive_train_cleaned_tokens_list = []
negative_train_cleaned_tokens_list = []
#unsup_train_cleaned_tokens_list = []

for tokens in positive_train_tweet_tokens:
    positive_train_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_train_tweet_tokens:
    negative_train_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
# for tokens in unsup_train_tweet_tokens:
#     unsup_train_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

#print(positive_cleaned_tokens_list)
# all_pos_words = get_all_words(positive_train_cleaned_tokens_list)
# #print(all_pos_words)
# freq_dist_pos = FreqDist(all_pos_words)
# print(freq_dist_pos.most_common(10))



positive_train_tokens_for_model = get_tweets_for_model(positive_train_cleaned_tokens_list)
negative_train_tokens_for_model = get_tweets_for_model(negative_train_cleaned_tokens_list)
# unsup_train_tokens_for_model = get_tweets_for_model(unsup_train_cleaned_tokens_list)

positive_train_dataset = [ (tweet_dict, "Positive") 
                      for tweet_dict in positive_train_tokens_for_model]
negative_train_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_train_tokens_for_model]
# unsup_train_dataset = [(tweet_dict, "Unsup")
#                      for tweet_dict in unsup_train_tokens_for_model]

dataset = positive_train_dataset + negative_train_dataset 

train_data = dataset


In [27]:
positive_test_tweets = test_text_pos
negative_test_tweets = test_text_neg
# text = twitter_samples.strings('tweets.20150430-223406.json')
#tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

positive_test_tweet_tokens = [word_tokenize(i) for i in positive_test_tweets]
negative_test_tweet_tokens = [word_tokenize(i) for i in negative_test_tweets]

positive_test_cleaned_tokens_list = []
negative_test_cleaned_tokens_list = []

for tokens in positive_test_tweet_tokens:
    positive_test_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_test_tweet_tokens:
    negative_test_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

positive_test_tokens_for_model = get_tweets_for_model(positive_test_cleaned_tokens_list)
negative_test_tokens_for_model = get_tweets_for_model(negative_test_cleaned_tokens_list)

positive_test_dataset = [ (tweet_dict, "Positive") 
                      for tweet_dict in positive_test_tokens_for_model]
negative_test_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_test_tokens_for_model]

dataset = positive_test_dataset + negative_test_dataset

test_data = dataset

In [29]:
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.614586818333891
Most Informative Features
                    2/10 = True           Negati : Positi =     76.3 : 1.0
                    4/10 = True           Negati : Positi =     64.5 : 1.0
                    *1/2 = True           Negati : Positi =     57.0 : 1.0
                    3/10 = True           Negati : Positi =     44.1 : 1.0
                    boll = True           Negati : Positi =     39.0 : 1.0
                     uwe = True           Negati : Positi =     37.6 : 1.0
                    7/10 = True           Positi : Negati =     33.2 : 1.0
                    8/10 = True           Positi : Negati =     27.6 : 1.0
             unwatchable = True           Negati : Positi =     27.0 : 1.0
                  awful. = True           Negati : Positi =     27.0 : 1.0
None
